Testando validação com um template específico

In [31]:
import pandas as pd

In [32]:
# campos de teste
campos = [
    {"nome": "servico_preco", "nulo": True, "tipo": "float"}, 
    {"nome": "mercadoria_preco", "nulo": False, "tipo": "int64"}, 
    {"nome": "produto_empresa", "nulo": False, "tipo": "float"}, 
    {"nome": "mercadoria_empresa", "nulo": True, "tipo": "int64"}, 
    {"nome": "item_titulo", "nulo": False, "tipo": "bool"}
]

In [33]:
# importando arquivo que vai ser testado com base nos campos
df = pd.read_csv('file.csv')
print(set(df.columns))
print(set(campo['nome'] for campo in campos))

{'item_titulo', 'servico_preco', 'produto_empresa', 'mercadoria_preco', 'mercadoria_empresa'}
{'item_titulo', 'servico_preco', 'produto_empresa', 'mercadoria_preco', 'mercadoria_empresa'}


obtendo itens do df e campos e fazendo verificação inicial

In [34]:
campos_df = set(df.columns)
campos_esperados = set(campo['nome'] for campo in campos)

print(df.dtypes)

campos_faltando = campos_df - campos_esperados

if(len(campos_df) > len(campos_esperados)):
    print(f"Arquivo com campo em excesso: {', '.join(campos_faltando)}")

if(len(campos_df) < len(campos_esperados)):
    print(f"Campos não esperados no arquivo: {', '.join(campos_faltando)}")

servico_preco          object
mercadoria_preco        int64
produto_empresa       float64
mercadoria_empresa     object
item_titulo              bool
dtype: object


função de comparação

In [35]:
def is_valid_datetime(string):
    try:
        pd.to_datetime(string)
        return True
    except ValueError:
        return False

def adicionar_erro(erros, mensagem):
    if mensagem not in erros:
        erros.append(mensagem)

loop principal

In [36]:
# supondo que para entrar aqui, todos os campos devem ser iguais em nomenclatura e numero
def validando():
    erros = []
    for campo in campos:
        nome_campo = campo['nome']
        nulo_esperado = campo['nulo']
        tipo_esperado = campo['tipo']

        if nome_campo not in campos_df:
            adicionar_erro(erros, f'Campo {nome_campo} não encontrado no dataframe')
            break
        
        nulos = df[nome_campo].isna().any()

        # negando o nulo para brincar com a tabela verdade
        if not nulo_esperado and nulos:
            adicionar_erro(erros, f"Campo '{nome_campo}' não deveria possuir valores nulos, mas possui.")
            break

        # inicio da verificação de tipagem de cada coluna
        for v in df[nome_campo]:
            if tipo_esperado == 'datetime64':
                if not is_valid_datetime(v):
                    adicionar_erro(erros, f"Campo '{nome_campo}' possuem valores que não são data.")
                    break
            if tipo_esperado == 'int64':
                if nulo_esperado:
                    if not isinstance(v, float):
                        adicionar_erro(erros, f"Campo '{nome_campo}' possuem valores que não são {tipo_esperado}.")
                else:
                    if not isinstance(v, int):
                        adicionar_erro(erros, f"Campo '{nome_campo}' possuem valores que não são {tipo_esperado}.")
            if tipo_esperado == 'float':
                # print(not isinstance(v, float), v, nome_campo)
                if not isinstance(v, float):
                    adicionar_erro(erros, f"Campo '{nome_campo}' possuem valores que não são {tipo_esperado}.")
            if tipo_esperado == 'bool':
                # print(not isinstance(v, bool), v, nome_campo)
                if not isinstance(v, bool):
                    adicionar_erro(erros, f"Campo '{nome_campo}' possuem valores que não são {tipo_esperado}.")
            if tipo_esperado == 'object':
                # print(not isinstance(v, str), v, nome_campo)
                if not isinstance(v, str):
                    adicionar_erro(erros, f"Campo '{nome_campo}' possuem valores que não são {tipo_esperado}.")

    if erros:
        return erros
    
    return None

validando()

["Campo 'servico_preco' possuem valores que não são float.",
 "Campo 'mercadoria_empresa' possuem valores que não são int64."]

In [37]:
def tenta_converter_com_pandas(valor, tipo):
    """Tenta converter o valor para o tipo especificado usando o pandas"""
    try:
        pd.Series([valor]).astype(tipo)
        return True
    except:
        return False

In [38]:
def verificar_dados(df, campos):
    """Verifica os dados de um dataframe com base em campos específicos. Utiliza um mapeamento de tipos para realizar a verificação."""

    try:
        # Iterando por cada linha e coluna
        for j in range(len(df.columns)):
            tipo_esperado = campos[j]['tipo']
            nome_campo = campos[j]['nome']
            print(tipo_esperado)
            for i in range(len(df)):
                data = df.iloc[i, j]
                dtype = type(data)
                

                if not tenta_converter_com_pandas(
                    data, tipo_esperado
                ):  # Se não for possível converter
                    raise Exception(
                        f"\nErro ao converter '{data}' para o tipo: {tipo_esperado}. \n Linha {i+1}, coluna '{nome_campo}'."
                    )
                
    except Exception as error:
        raise Exception("Erro ao verificar dados: " + error.args[0])

verificar_dados(df, campos)


float


Exception: Erro ao verificar dados: 
Erro ao converter 'asd' para o tipo: float. 
 Linha 6, coluna 'servico_preco'.